In [3]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, CLIPProcessor, CLIPModel
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.44it/s]


In [2]:
model.vision_tower

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(577, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=

In [5]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336")
clip_model.vision_model

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(577, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [6]:
import lovely_tensors as lt
lt.monkey_patch()

In [15]:
zm = clip_model.vision_model.to(dtype=torch.float16)
zm

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(577, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [17]:
model.vision_tower.vision_model

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(577, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [28]:
# check if zm and model.vision_tower.vision_model have the same weights
for i, (p1, p2) in enumerate(zip(zm.parameters(), model.vision_tower.vision_model.parameters())):
    if not torch.allclose(p1, p2, rtol=1e-4, atol=1e-2):
        print(i)
        print(p1)
        print(p2)

110
Parameter containing:
Parameter[1024] f16 2Kb x∈[-1.825, 5.012] μ=0.001 σ=0.237 grad
Parameter containing:
Parameter[1024] f16 2Kb x∈[-1.828, 5.000] μ=0.001 σ=0.237 grad
180
Parameter containing:
Parameter[1024] f16 2Kb x∈[-4.453, 1.655] μ=-0.025 σ=0.283 grad
Parameter containing:
Parameter[1024] f16 2Kb x∈[-4.438, 1.656] μ=-0.025 σ=0.283 grad
196
Parameter containing:
Parameter[1024] f16 2Kb x∈[-5.793, 1.517] μ=-0.028 σ=0.317 grad
Parameter containing:
Parameter[1024] f16 2Kb x∈[-5.781, 1.516] μ=-0.028 σ=0.317 grad
262
Parameter containing:
Parameter[1024] f16 2Kb x∈[-4.387, 4.863] μ=0.013 σ=0.493 grad
Parameter containing:
Parameter[1024] f16 2Kb x∈[-4.375, 4.875] μ=0.013 σ=0.493 grad
294
Parameter containing:
Parameter[1024] f16 2Kb x∈[-6.293, 11.375] μ=-0.014 σ=1.055 grad
Parameter containing:
Parameter[1024] f16 2Kb x∈[-6.281, 11.375] μ=-0.014 σ=1.055 grad
310
Parameter containing:
Parameter[1024] f16 2Kb x∈[-4.762, 5.621] μ=-0.014 σ=0.489 grad
Parameter containing:
Parameter[